<a href="https://colab.research.google.com/github/GmBeHappy/JSTP_360_Camera_Human_Detection/blob/main/%E0%B9%8BJSTP_360_degree_human_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

import numpy as np
import tensorflow.compat.v1 as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import cv2
import time
from google.colab.patches import cv2_imshow

Num GPUs Available:  1


In [3]:
class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

In [25]:
if __name__ == "__main__":
    model_path = '/content/drive/MyDrive/faster_rcnn_nas_coco_2018_01_28/frozen_inference_graph.pb'
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.7
    cap = cv2.VideoCapture('/content/drive/MyDrive/jstp-test-video/test2560x1280.mp4')
    out = cv2.VideoWriter('outputInf2560x1280.mp4', 0x7634706d, 20.0, (2560,1280))
    while True:
        ret, img = cap.read()
        if ret==True:
            # img = cv2.resize(img, (1280, 720))
            boxes, scores, classes, num = odapi.processFrame(img)

            # Visualization of the results of a detection.

            for i in range(len(boxes)):
                # Class 1 represents human
                if classes[i] == 1 and scores[i] > threshold:
                    box = boxes[i]
                    cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)

            # cv2_imshow(img)
            out.write(img)
            # key = cv2.waitKey(1)
            # if key & 0xFF == ord('q'):
            #     break
        else:
          break
  
    out.release()
    cv2.destroyAllWindows()
        
        

Elapsed Time: 7.043079376220703
Elapsed Time: 0.9628012180328369
Elapsed Time: 0.968794584274292
Elapsed Time: 0.959789514541626
Elapsed Time: 0.9620907306671143
Elapsed Time: 0.9614229202270508
Elapsed Time: 0.9606482982635498
Elapsed Time: 0.9622499942779541
Elapsed Time: 0.9647603034973145
Elapsed Time: 0.9611368179321289
Elapsed Time: 0.9612438678741455
Elapsed Time: 0.9627814292907715
Elapsed Time: 0.9662160873413086
Elapsed Time: 0.9670662879943848
Elapsed Time: 0.9615473747253418
Elapsed Time: 0.9642078876495361
Elapsed Time: 0.961580753326416
Elapsed Time: 0.962313175201416
Elapsed Time: 0.9617795944213867
Elapsed Time: 0.9629974365234375
Elapsed Time: 0.961646318435669
Elapsed Time: 0.961529016494751
Elapsed Time: 0.9651775360107422
Elapsed Time: 0.9604401588439941
Elapsed Time: 0.9604802131652832
Elapsed Time: 0.9650237560272217
Elapsed Time: 0.9635672569274902
Elapsed Time: 0.9612922668457031
Elapsed Time: 0.9654347896575928
Elapsed Time: 0.9630310535430908
Elapsed Time: 0.9